In [3]:
## module needed

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import librosa
import os

import pickle

In [3]:
## function for splitting the song into equal size (1000)

def cut_song(song):
    start = 0
    end = len(song)

    song_pieces = []

    while start + 100000 < end:
        song_pieces.append(song[start:start+100000])

    start += 100000

    return song_pieces

In [42]:
## function for converting into melspectrograms

def prepare_song(song_path):
    list_matrices = []
    y,sr = librosa.load(song_path,sr=22050)
    song_pieces = cut_song(y)
    for song_piece in song_pieces:
        melspect = librosa.feature.melspectrogram(song_piece)
        list_matrices.append(melspect)
    return list_matrices

In [10]:
## getting all the genre

genre = os.listdir('tracks')
print(genre)

['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']


In [26]:
X = []
y = []

for i in range(len(genre)):
    for song_name in os.listdir(os.path.join('tracks',genre[i])):
        try:
            song_pieces = prepare_song(os.path.join('tracks', genre[i], song_name))
            X += song_pieces
            y += ([i]*len(song_pieces))
        except Exception:
            print(f"unable to add this file {os.path.join('tracks', genre[i], song_name)}")
        
    print(f"{i+1}/{len(genre)} done")

1/10 done
2/10 done
3/10 done
4/10 done
5/10 done


/home/sunil/.local/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


unable to add this file tracks/jazz/jazz.00054.wav
6/10 done
7/10 done
8/10 done
9/10 done
10/10 done


In [30]:
## preserving the X and y

with open('X.pickle', 'wb') as f:
    pickle.dump(X, f)

with open('y.pickle', 'wb') as f:
    pickle.dump(y, f)

In [12]:
## loading the data back

X = pickle.load(open('X.pickle', 'rb'))
X = np.array(X).reshape(-1, 128, 196, 1)
y = pickle.load(open('y.pickle', 'rb'))
y = np.array(y)

## train / testing

In [1]:
## packages

import numpy as np
from sklearn.model_selection import train_test_split


import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y,
                                                    test_size=0.33,
                                                    random_state=42)

X_val, X_test, y_val, y_test = train_test_split(X_test, 
                                                y_test,
                                                test_size=0.5,
                                                random_state=42)

In [6]:
## model design

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 196, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))


model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))
# since ther are 10 labels

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 194, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 97, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 95, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 47, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 45, 64)        36928     
                                                                 
 flatten (Flatten)           (None, 80640)             0

In [7]:

from tensorflow.keras.optimizers import RMSprop
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics='accuracy')

history = model.fit(X_train, y_train, epochs=10, 
                    validation_data=(X_val, y_val))

Epoch 1/10
126/126 [==============================] - 112s 882ms/step - loss: -41.4381 - accuracy: 0.1220 - val_loss: -37.7128 - val_accuracy: 0.1345
Epoch 2/10
126/126 [==============================] - 107s 847ms/step - loss: -41.9157 - accuracy: 0.1220 - val_loss: -37.7128 - val_accuracy: 0.1345
Epoch 3/10
126/126 [==============================] - 110s 874ms/step - loss: -41.9157 - accuracy: 0.1220 - val_loss: -37.7128 - val_accuracy: 0.1345
Epoch 4/10
126/126 [==============================] - 108s 861ms/step - loss: -41.9157 - accuracy: 0.1220 - val_loss: -37.7128 - val_accuracy: 0.1345
Epoch 5/10
126/126 [==============================] - 106s 844ms/step - loss: -41.9157 - accuracy: 0.1220 - val_loss: -37.7128 - val_accuracy: 0.1345
Epoch 6/10
126/126 [==============================] - 100s 796ms/step - loss: -41.9157 - accuracy: 0.1220 - val_loss: -37.7128 - val_accuracy: 0.1345
Epoch 7/10
126/126 [==============================] - 21104s 169s/step - loss: -41.9157 - accuracy: 

In [16]:

model.predict(X[5])

ValueError: in user code:

    File "c:\Users\sunil\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1801, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\sunil\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1790, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\sunil\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1783, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\sunil\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1751, in predict_step
        return self(x, training=False)
    File "c:\Users\sunil\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\suni